In [ ]:
import os
from dotenv import load_dotenv
from langchain.text_splitter import MarkdownTextSplitter
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
# 加载 .env 文件中的环境变量
load_dotenv()

# 从 .env 文件中获取密钥和 API 基础地址
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("BASE_URL")  # 默认使用官方 URL

# 初始化 OpenAIEmbeddings
embeddings = OpenAIEmbeddings(api_key=api_key, base_url=base_url)

# 文件路径
file_path = "circosjsRM.md"
index_path = "circosjs_index"

# 检查索引是否存在
if os.path.exists(index_path):
    print("加载已有的向量索引...")
    try:
        # 允许危险反序列化，仅在信任数据来源时启用
        faiss_index = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
        print("索引加载成功！")
    except (ValueError, IOError) as e:
        print(f"加载索引失败: {e}")
        print("重新生成索引...")
        os.remove(index_path)  # 删除旧索引
        # 重新创建索引
        with open(file_path, "r", encoding="utf-8") as file:
            markdown_text = file.read()
        splitter = MarkdownTextSplitter(chunk_size=500, chunk_overlap=50)
        chunks = splitter.split_text(markdown_text)
        faiss_index = FAISS.from_texts(chunks, embeddings)
        faiss_index.save_local(index_path)
else:
    print("未找到索引，创建新索引...")
    # 创建新索引
    with open(file_path, "r", encoding="utf-8") as file:
        markdown_text = file.read()
    splitter = MarkdownTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_text(markdown_text)
    faiss_index = FAISS.from_texts(chunks, embeddings)
    faiss_index.save_local(index_path)

# 初始化 LLM
llm = OpenAI(
    api_key=api_key,
    base_url=base_url,
    model="gpt-3.5-turbo",
    temperature=0.7,
    streaming=True
)
print(f"Base URL: {base_url}")
print(f"API Key: {api_key}")
try:
    test_response = llm.invoke("Hello")
    print("测试响应:", test_response)
    #打印baseurl和apikey

except Exception as e:
    print("测试错误:", e)
except requests.exceptions.RequestException as e:
    print(f"API 请求错误: {e}")

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=faiss_index.as_retriever(),
    chain_type="stuff",
    return_source_documents=True,  # 返回检索到的文档
    verbose=True
)

# 定义调用函数
def generate_code_from_query(query):
    try:
        try:
            response = qa_chain.invoke(query)
        except Exception as e:
            print(f"调用检索链时出错: {e}")
        print("\n检索到的相关文档片段:")
        for doc in response['source_documents']:
            print(doc.page_content)
        return response['result']
    except Exception as e:
        print(f"Error during code generation: {e}")
        if hasattr(e, 'response') and hasattr(e.response, 'text'):
            print(f"详细错误信息: {e.response.text}")
        return None

# 示例调用
if __name__ == "__main__":
    user_query = "画一个环形柱状图的track，深绿色"
    generated_code = generate_code_from_query(user_query)
    print("\n生成的代码:")
    print(generated_code)
